In [2]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

# Crear la sesión de Spark
spark = SparkSession.builder.appName('ejercicio_3').getOrCreate()

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
24/09/21 16:39:52 WARN Utils: Your hostname, Andres resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/09/21 16:39:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/21 16:39:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Leer los archivos
df_cundinamarca = spark.read.csv('data/deptos/25_CUNDINAMARCA_R1.csv', header=True, inferSchema=True, encoding='UTF-8')
df_internet = spark.read.csv('data/Internet_Fijo_Penetraci_n_Municipio.csv', header=True, inferSchema=True, encoding='UTF-8')

In [4]:
# Convertir el campo DEPARTAMENTO y COD_DEPARTAMENTO a string en ambos datasets para que coincidan
df_cundinamarca = df_cundinamarca.withColumn('DEPARTAMENTO', F.col('DEPARTAMENTO').cast('string'))
df_internet = df_internet.withColumn('COD_DEPARTAMENTO', F.col('COD_DEPARTAMENTO').cast('string'))

In [5]:
# Renombrar las columnas en df_cundinamarca para evitar conflictos
df_cundinamarca = df_cundinamarca.select([F.col(c).alias(f"cundi_{c}") for c in df_cundinamarca.columns])

In [6]:
# Escapar columnas con caracteres especiales en df_internet
df_internet = df_internet.select([F.col(f"`{c}`").alias(f"int_{c.replace(' ', '_').replace('.', '_')}") for c in df_internet.columns])

In [7]:
# Realizar un join filtrando por el código de departamento de Cundinamarca (25)
df_cundinamarca_joined = df_cundinamarca.join(
    df_internet.filter(F.col('int_COD_DEPARTAMENTO') == '25'),
    df_cundinamarca['cundi_DEPARTAMENTO'] == df_internet['int_COD_DEPARTAMENTO'],
    how='inner'
)

In [7]:
# Mostrar algunas filas en el editor para comprobar
df_cundinamarca_joined.show()

+------------------+---------------+--------------------+---------------+-----------------------+------------------+---------------------+-----------------------------+-------+-------------+--------------------+----------------+-----------------+--------------------+--------------------------------+------------------+----------+
|cundi_DEPARTAMENTO|cundi_MUNICIPIO|cundi_NUMERO_PREDIAL|cundi_DIRECCION|cundi_DESTINO_ECONOMICO|cundi_AREA_TERRENO|cundi_AREA_CONSTRUIDA|cundi_NUMERO_PREDIAL_ANTERIOR|int_AÑO|int_TRIMESTRE|int_COD_DEPARTAMENTO|int_DEPARTAMENTO|int_COD_MUNICIPIO|       int_MUNICIPIO|int_No__ACCESOS_FIJOS_A_INTERNET|int_POBLACIÓN_DANE|int_INDICE|
+------------------+---------------+--------------------+---------------+-----------------------+------------------+---------------------+-----------------------------+-------+-------------+--------------------+----------------+-----------------+--------------------+--------------------------------+------------------+----------+
|      

In [8]:
# Exportar el resultado para evitar problemas con el editor
output_path = "data/deptos/transformaciones/cundinamarca_joined.csv"
df_cundinamarca_joined.coalesce(1).write.csv(output_path, header=True, mode='overwrite')

In [9]:
# Cerrar la sesión de Spark al finalizar
spark.stop()

In [8]:
df_cundinamarca_joined.printSchema()

root
 |-- cundi_DEPARTAMENTO: string (nullable = true)
 |-- cundi_MUNICIPIO: integer (nullable = true)
 |-- cundi_NUMERO_PREDIAL: decimal(30,0) (nullable = true)
 |-- cundi_DIRECCION: string (nullable = true)
 |-- cundi_DESTINO_ECONOMICO: string (nullable = true)
 |-- cundi_AREA_TERRENO: integer (nullable = true)
 |-- cundi_AREA_CONSTRUIDA: integer (nullable = true)
 |-- cundi_NUMERO_PREDIAL_ANTERIOR: decimal(20,0) (nullable = true)
 |-- int_AÑO: integer (nullable = true)
 |-- int_TRIMESTRE: integer (nullable = true)
 |-- int_COD_DEPARTAMENTO: string (nullable = true)
 |-- int_DEPARTAMENTO: string (nullable = true)
 |-- int_COD_MUNICIPIO: integer (nullable = true)
 |-- int_MUNICIPIO: string (nullable = true)
 |-- int_No__ACCESOS_FIJOS_A_INTERNET: integer (nullable = true)
 |-- int_POBLACIÓN_DANE: integer (nullable = true)
 |-- int_INDICE: string (nullable = true)



In [9]:
# Mostrar algunas filas en el editor para comprobar
df_cundinamarca_joined.show(1)

+------------------+---------------+--------------------+---------------+-----------------------+------------------+---------------------+-----------------------------+-------+-------------+--------------------+----------------+-----------------+-------------+--------------------------------+------------------+----------+
|cundi_DEPARTAMENTO|cundi_MUNICIPIO|cundi_NUMERO_PREDIAL|cundi_DIRECCION|cundi_DESTINO_ECONOMICO|cundi_AREA_TERRENO|cundi_AREA_CONSTRUIDA|cundi_NUMERO_PREDIAL_ANTERIOR|int_AÑO|int_TRIMESTRE|int_COD_DEPARTAMENTO|int_DEPARTAMENTO|int_COD_MUNICIPIO|int_MUNICIPIO|int_No__ACCESOS_FIJOS_A_INTERNET|int_POBLACIÓN_DANE|int_INDICE|
+------------------+---------------+--------------------+---------------+-----------------------+------------------+---------------------+-----------------------------+-------+-------------+--------------------+----------------+-----------------+-------------+--------------------------------+------------------+----------+
|                25|        

In [10]:
# Verificar si cundi_DEPARTAMENTO es igual a int_DEPARTAMENTO
df_cundinamarca_joined.filter(F.col('cundi_DEPARTAMENTO') != F.col('int_DEPARTAMENTO')).select('cundi_DEPARTAMENTO', 'int_DEPARTAMENTO').show(10, truncate=False)

# Verificar si cundi_MUNICIPIO es igual a int_MUNICIPIO
df_cundinamarca_joined.filter(F.col('cundi_MUNICIPIO') != F.col('int_COD_MUNICIPIO')).select('cundi_MUNICIPIO', 'int_COD_MUNICIPIO').show(10, truncate=False)


+------------------+----------------+
|cundi_DEPARTAMENTO|int_DEPARTAMENTO|
+------------------+----------------+
|25                |CUNDINAMARCA    |
|25                |CUNDINAMARCA    |
|25                |CUNDINAMARCA    |
|25                |CUNDINAMARCA    |
|25                |CUNDINAMARCA    |
|25                |CUNDINAMARCA    |
|25                |CUNDINAMARCA    |
|25                |CUNDINAMARCA    |
|25                |CUNDINAMARCA    |
|25                |CUNDINAMARCA    |
+------------------+----------------+
only showing top 10 rows

+---------------+-----------------+
|cundi_MUNICIPIO|int_COD_MUNICIPIO|
+---------------+-----------------+
|25001          |25851            |
|25001          |25488            |
|25001          |25769            |
|25001          |25148            |
|25001          |25181            |
|25001          |25151            |
|25001          |25799            |
|25001          |25662            |
|25001          |25436            |
|25001    

In [12]:
# Filtrar e imprimir las filas donde int_MUNICIPIO sea igual a 25001
df_cundinamarca_joined.filter(F.col('int_COD_MUNICIPIO') == 11001).select('cundi_MUNICIPIO', 'int_COD_MUNICIPIO', 'int_MUNICIPIO', 'int_POBLACIÓN_DANE').show(100, truncate=False)


+---------------+-----------------+-------------+------------------+
|cundi_MUNICIPIO|int_COD_MUNICIPIO|int_MUNICIPIO|int_POBLACIÓN_DANE|
+---------------+-----------------+-------------+------------------+
+---------------+-----------------+-------------+------------------+



In [21]:
from pyspark.sql import functions as F

# 1. Leer los archivos
df_cundinamarca = spark.read.csv('data/deptos/25_CUNDINAMARCA_R1.csv', header=True, inferSchema=True, encoding='UTF-8')
df_internet = spark.read.csv('data/Internet_Fijo_Penetraci_n_Municipio.csv', header=True, inferSchema=True, encoding='UTF-8')

# 2. Renombrar la columna MUNICIPIO a NOMBRE_MUNICIPIO en df_internet para evitar conflictos
df_internet = df_internet.withColumnRenamed("MUNICIPIO", "NOMBRE_MUNICIPIO")

# 3. Renombrar la columna DEPARTAMENTO en df_internet para evitar ambigüedad
df_internet = df_internet.withColumnRenamed("DEPARTAMENTO", "INT_DEPARTAMENTO")

# 4. Renombrar columnas con espacios y caracteres especiales en df_internet
df_internet = df_internet.withColumnRenamed("POBLACIÓN DANE", "POBLACION_DANE") \
    .withColumnRenamed("No. ACCESOS FIJOS A INTERNET", "ACCESOS_FIJOS_A_INTERNET")

# 5. Realizar el join usando el código del municipio como clave
df_joined = df_cundinamarca.join(
    df_internet,
    df_cundinamarca["MUNICIPIO"] == df_internet["COD_MUNICIPIO"],
    how="inner"
)

# 6. Eliminar duplicados basados en el código del municipio
df_joined = df_joined.dropDuplicates(["MUNICIPIO", "NOMBRE_MUNICIPIO", "DEPARTAMENTO", "POBLACION_DANE"])

# 7. Seleccionar y mostrar el código del municipio, su nombre, y el departamento
df_joined.select('MUNICIPIO', 'NOMBRE_MUNICIPIO', 'DEPARTAMENTO', 'POBLACION_DANE').show(100, truncate=False)

# 8. Contar el número de municipios únicos en Cundinamarca
num_municipios_unicos = df_joined.select('MUNICIPIO').distinct().count()
num_municipios_unicos

+---------+----------------+------------+--------------+
|MUNICIPIO|NOMBRE_MUNICIPIO|DEPARTAMENTO|POBLACION_DANE|
+---------+----------------+------------+--------------+
|25001    |AGUA DE DIOS    |25          |12813         |
|25001    |AGUA DE DIOS    |25          |11966         |
|25120    |CABRERA         |25          |5263          |
|25120    |CABRERA         |25          |4885          |
|25175    |CHIA            |25          |149570        |
|25099    |BOJACA          |25          |11898         |
|25001    |AGUA DE DIOS    |25          |12325         |
|25148    |CAPARRAPI       |25          |12990         |
|25148    |CAPARRAPI       |25          |13506         |
|25086    |BELTRAN         |25          |1960          |
|25148    |CAPARRAPI       |25          |13931         |
|25099    |BOJACA          |25          |11535         |
|25086    |BELTRAN         |25          |1885          |
|25175    |CHIA            |25          |141308        |
|25120    |CABRERA         |25 

31

In [24]:
# Leer el archivo original de Cundinamarca
df_cundinamarca_original = spark.read.csv('data/deptos/CUNDINAMARCA_REG2.csv', header=True, inferSchema=True, encoding='UTF-8')

# Verificar si Bogotá está en el dataset original de Cundinamarca usando el código 11001
df_bogota_cundinamarca = df_cundinamarca_original.filter(F.col("MUNICIPIO") == "11001")

# Mostrar los resultados
df_bogota_cundinamarca.show(truncate=False)


+------------+---------+--------------+-------------+----------------+--------------+-------------+----------------+--------------+--------------+-------+---------+-------+--------------+-----+---------+-----------------+--------------+-------+---------+-------+--------------+-----+---------+-----------------+--------------+-------+---------+-------+--------------+-----+---------+-----------------+-----------------------+
|DEPARTAMENTO|MUNICIPIO|NUMERO_PREDIAL|ZONA_FISICA_1|ZONA_ECONOMICA_1|AREA_TERRENO_1|ZONA_FISICA_2|ZONA_ECONOMICA_2|AREA_TERRENO_2|HABITACIONES_1|BANOS_1|LOCALES_1|PISOS_1|TIPIFICACION_1|USO_1|PUNTAJE_1|AREA_CONSTRUIDA_1|HABITACIONES_2|BANOS_2|LOCALES_2|PISOS_2|TIPIFICACION_2|USO_2|PUNTAJE_2|AREA_CONSTRUIDA_2|HABITACIONES_3|BANOS_3|LOCALES_3|PISOS_3|TIPIFICACION_3|USO_3|PUNTAJE_3|AREA_CONSTRUIDA_3|NUMERO_PREDIAL_ANTERIOR|
+------------+---------+--------------+-------------+----------------+--------------+-------------+----------------+--------------+--------------+--

In [25]:
#cerrar la sesión de Spark
spark.stop()